In [ ]:
# default_exp nlp.fasttext

# FastText - UMAP - HDBSCAN

> API details.

In [ ]:
import os
import time
import sys
import inspect

import pandas as pd
import numpy as np

import datetime

try:
    from fasttext import train_unsupervised
    import fasttext
except:
    from fastText import train_unsupervised
    import fastText
    
import umap.umap_ as umap
import hdbscan

import plotly.graph_objs as go
import plotly.offline as py
import plotly.graph_objs as go
from IPython.display import Image,display

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Get Data

In [ ]:
fasttext_data_folder = "../data/fasttext_data"
train_data = os.path.join(fasttext_data_folder,"text_file.text")

In [ ]:
text_df = pd.read_csv(train_data, names=["itemDesc"])
text_df.head()

,itemDesc
0,MINI DONUTS FOURRES ASSORTI X6
1,NAVETTE ASSORTIMENT X30
2,BRIOCHE MOUNA 400G
3,BRIOCHETTE ST GENIX X4 LOCAL
4,BRIOCHE ST GENIX LOCAL


## FastText training

In [ ]:
cbow_model = os.path.join(fasttext_data_folder,"model_cbow.bin")
skipgram_model = os.path.join(fasttext_data_folder,"model_skipgram.bin")


**train_unsupervised parameters**
    * input             # training file path (required)
    lr                # learning rate [0.1]
    dim               # size of word vectors [100]
    ws                # size of the context window [5]
    epoch             # number of epochs [5]
    minCount          # minimal number of word occurences [1]
    minCountLabel     # minimal number of label occurences [1]
    minn              # min length of char ngram [0]
    maxn              # max length of char ngram [0]
    neg               # number of negatives sampled [5]
    wordNgrams        # max length of word ngram [1]
    loss              # loss function {ns, hs, softmax, ova} [softmax]
    bucket            # number of buckets [2000000]
    thread            # number of threads [number of cpus]
    lrUpdateRate      # change the rate of updates for the learning rate [100]
    t                 # sampling threshold [0.0001]
    label             # label prefix ['__label__']
    verbose           # verbose [2]
    pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []

In [ ]:
model_to_use = "skipgram"

if model_to_use == "skipgram":
    # Skipgram model :
    model_skipgram = train_unsupervised(train_data, model='skipgram')
    model_skipgram.save_model(skipgram_model)

else:
    #Cbow model :
    model_cbow = train_unsupervised(train_data, model='cbow')
    model_cbow.save_model(cbow_model)

In [ ]:
# Loading of the fastext pretained model
model_skipgram=fasttext.load_model(skipgram_model)

In [ ]:
model_skipgram.get_dimension()

100

In [ ]:
# The get_sentence_vector takes all vectors for all the words in the query, divide each of them by their respective norm, and then average all vectors together
def query_to_vector(col_query, model_fastText):
    vector = col_query.apply(lambda x:model_fastText.get_sentence_vector(x.replace('\n',' ')))
    return vector

In [ ]:
%timeit text_df['vector'] = query_to_vector(text_df['itemDesc'], model_skipgram)
del model_skipgram # We do not need the pretrained-vector in memory

The slowest run took 4.69 times longer than the fastest. This could mean that an intermediate result is being cached.
20.5 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
text_df.head(10)

,itemDesc,vector
0,MINI DONUTS FOURRES ASSORTI X6,"[0.021459619, 0.10777775, -0.015001227, -0.022..."
1,NAVETTE ASSORTIMENT X30,"[0.014762469, 0.046633586, -0.042754717, -0.01..."
2,BRIOCHE MOUNA 400G,"[-0.052082933, 0.0019508253, 0.006112094, -0.0..."
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.010170468, -0.015941408, -0.0090812, -0.02..."
4,BRIOCHE ST GENIX LOCAL,"[-0.017857304, -0.030833643, -0.0024532434, -0..."
5,NAVETTE X20,"[0.026398266, -0.0045050173, -0.047125924, -0...."
6,TARTE AU SUCRE 6P LOCAL,"[-0.02277725, -0.0873375, -0.012378287, 0.0324..."
7,MINI SUISSE X8,"[0.09523194, 0.010796974, 0.14384931, -0.13409..."
8,4 P RAIS+4 P CHOC.+4 CROIS. PB,"[0.011881388, 0.034624543, -0.043392614, 0.031..."
9,X5 POCHE TALOAK,"[0.011327958, 0.00809449, 0.006738869, -0.0332..."


## Dimension reduction with UMAP

In [ ]:
# We regroup all the vectors as a numpy array
vecs=text_df.vector.values
vecs=np.stack(vecs, axis=0)
vecs.shape

(583, 100)

In [ ]:
col_names = ["FT_"+str(x) for x in range(0,100)]
print(len(col_names))
test = pd.DataFrame(vecs, columns=col_names)

100


In [ ]:
fit = umap.UMAP(n_neighbors=30,min_dist=0.1,n_components=3,metric='cosine',random_state=42)
%time u = fit.fit_transform(vecs)

text_df['x']=u[:,0]
text_df['y']=u[:,1]
text_df['z']=u[:,2]

CPU times: user 1.43 s, sys: 59.1 ms, total: 1.49 s
Wall time: 1.4 s


In [ ]:
u.shape

(583, 3)

## 3D Vizualisation

In [ ]:
# This is the function to plot the queries in the embedding space.
# Here we reduce the embedding to a 3 dimensionnal space
def plot_cluster(df,iscolored=False,name='',interactive=True):
    
    if interactive:
        if iscolored:
            color=df['cluster'].values
        else: color = df['x']
        trace1 = go.Scatter3d(
            x=df['x'],
            y=df['y'],
            z=df['z'],

            mode='markers',
            marker=dict(
                size=3,
                color=color,                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.3
            ),
            text=color
        )

        data = [trace1]
        layout = go.Layout(
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            )
        )
        fig = go.Figure(data=data, layout=layout)
        file='../data/fasttext_data/'+name+'.html'
        py.iplot(fig, filename=file)
        py.plot(fig, filename=file,auto_open=False)

In [ ]:
is_interactive=False
filename='umap_embedding_description'

if is_interactive:
    plot_cluster(text_df,False,filename)

## Clustering with HDBSCAN

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,min_samples=5)

In [ ]:
clusterer

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='euclidean', min_cluster_size=5, min_samples=5, p=None,
        prediction_data=False)

In [ ]:
%time text_df['cluster']= clusterer.fit_predict(u)

CPU times: user 26 ms, sys: 5.75 ms, total: 31.8 ms
Wall time: 33.9 ms


In [ ]:
text_df.groupby(['cluster'])['itemDesc'].apply(list).apply(len).sort_values(ascending=False)[0:20]

cluster
-1     153
 13     95
 16     29
 1      28
 18     23
 8      22
 2      19
 25     19
 23     17
 9      16
 21     14
 3      14
 4      13
 5      13
 19     12
 6      11
 14     10
 10      9
 22      8
 12      8
Name: itemDesc, dtype: int64

In [ ]:
# The cluster number -1 means actually that the algorithm considered it as noise.
# So we will remove the noise
denoised = text_df[text_df.cluster!=-1]

In [ ]:
denoised.head()

,itemDesc,vector,x,y,z,cluster
0,MINI DONUTS FOURRES ASSORTI X6,"[0.021459619, 0.10777775, -0.015001227, -0.022...",1.819945,8.172493,5.891437,8
2,BRIOCHE MOUNA 400G,"[-0.052082933, 0.0019508253, 0.006112094, -0.0...",5.309334,5.404963,3.979075,23
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.010170468, -0.015941408, -0.0090812, -0.02...",5.848576,6.289163,5.398617,18
4,BRIOCHE ST GENIX LOCAL,"[-0.017857304, -0.030833643, -0.0024532434, -0...",5.889108,6.089911,5.217195,18
7,MINI SUISSE X8,"[0.09523194, 0.010796974, 0.14384931, -0.13409...",2.519321,7.063348,5.902770,10


In [ ]:
# Number of clusters
print('Total number of clusters: '+str(len(denoised.cluster.unique())))

Total number of clusters: 26


In [ ]:
clusters=denoised.groupby(['cluster'])['itemDesc'].apply(list)

In [ ]:
cluster_i = clusters[0]
print(len(cluster_i))
cluster_i

6


['4 MICRO DONUTS ROSE',
 '4 MICRO DONUTS CHOCO',
 '4 MICRO DONUTS BLANC CONFETTI',
 '4 MICRO DONUTS CHOCO POPPIES',
 '4 MICRO DONUTS ROSE POPPIES',
 '4 MICRO DONUTS BLANC CONFETTI']

## TF-IDF into the clusters

In [ ]:
def get_top_grams(cluster_j):
    tokens = nltk.word_tokenize(" ".join(cluster_j))

    #Create your bigrams
    tgs = nltk.trigrams(tokens)
    bgs = nltk.bigrams(tokens)

    #compute frequency distribution for all the bigrams in the text
    fdist = nltk.FreqDist(tgs)
    trigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
    fdist = nltk.FreqDist(bgs)
    bigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
    fdist = nltk.FreqDist(tokens)
    onegram = fdist.most_common(1)[0][0],fdist.most_common(1)[0][1]

    return trigram, bigram, onegram

In [ ]:
# Cluster description
for i, cluster_i in enumerate(clusters):
    print("---------")
    print("cluster {} size:{}".format(i,len(cluster_i)))
    print(get_top_grams(cluster_i))

---------
cluster 0 size:6


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(('X 4 LOCAL', 3), ('LOCAL COQUILLE', 6), ('LOCAL', 18))


In [ ]:
tokens = nltk.word_tokenize(" ".join(cluster_i))

#Create your bigrams
tgs = nltk.trigrams(tokens)
bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(tgs)
trigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
fdist = nltk.FreqDist(bgs)
bigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
fdist = nltk.FreqDist(tokens)
onegram = fdist.most_common(1)[0][0],fdist.most_common(1)[0][1]

print(trigram, bigram, onegram)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

('X 4 LOCAL', 3) ('LOCAL COQUILLE', 6) ('LOCAL', 18)


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_connectors.gcp.ipynb.
Converted 01_nlp.fasttext.ipynb.
Converted 02_forecasting.dataprep.ipynb.
Converted 03_models.catboost.ipynb.
Converted 04_nlp.nbsvm.ipynb.
Converted index.ipynb.
